In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import glob

read files

In [4]:
web_pages_dir = '../../personalisation_project/TVNZ/*/*'
files = glob.glob(web_pages_dir)

scraping

In [5]:
def fetch_meta(soup):
    image = soup.select_one('.Hero-image-inner img').attrs['src'].strip()
    content_container = soup.select_one('.Hero-container-content-inner')
    title = content_container.select_one('.Hero-title a').text.strip()
    description = content_container.select_one('.Hero-description').text.strip()

    return {
        'title': title,
        'description': description,
        'image_url': image
    }


def fetch_episodes(soup):
    episodes = soup.select_one('.Episodes-list')

    # for episode in episodes:
    #     episode.select_one('img')


In [6]:
%%time

data = []

for file in files[0:1]:
    splitted = file.split('/')
    category = splitted[4]
    file_path = '/'.join(splitted[4:])

    soup = BeautifulSoup(open(file), 'html.parser')

    content = fetch_meta(soup)
    content['category'] = category

    print(file_path)
    print(content)

documentary/proof.html
{'title': 'Proof', 'description': "Award-winning journalist, Guyon Espiner, investigates New Zealand's alcohol industry and drinking culture.", 'image_url': 'https://play-images-prod-ctf.tech.tvnz.co.nz/api/v1/web/image/1HLvx9bsvGSw78TmOSsHNn/3351481b423082f88b3057cfe1944091/OD_Proof_coverimg.png.3351481b423082f88b3057cfe1944091.jpg?width=1600&height=457', 'category': 'documentary'}
CPU times: user 29 ms, sys: 1.8 ms, total: 30.8 ms
Wall time: 30.2 ms
